In [80]:
import numpy as np
from astropy.io import fits
from marvin.tools import Maps
from tqdm import tqdm
import Bicone_Classifier
import multiprocessing as mp
import copy
import matplotlib.pyplot as plt
from marvin.utils.general.images import showImage
from marvin.tools.image import Image

from scipy import ndimage

In [3]:
HEADER_PATH = '/Users/runquanguan/Documents/dapall-v3_1_1-3.1.0.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']
petro_angle = hdu['NSA_ELPETRO_PHI']
sersic_angle = hdu['NSA_SERSIC_PHI']

In [49]:
#data = '9090-1901'

HEADER_PATH = '/Users/runquanguan/Documents/dapall-v3_1_1-3.1.0.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']
petro_angle = hdu['NSA_ELPETRO_PHI']
sersic_angle = hdu['NSA_SERSIC_PHI']

def plot_major_axis(data):
    


    dapall_index = list(plateifu).index(data)
    p_slope = np.tan(petro_angle[dapall_index])
    s_slope = np.tan(sersic_angle[dapall_index])

    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    st_vel = maps.stellar_vel.value
    oiii_ew = maps.emline_gew_oiii_5008.value
    
    
    
    # point-slope form
    slope = p_slope
    x_1 = st_vel.shape[0]/2
    y_1 = st_vel.shape[1]/2
    x = np.linspace(0, st_vel.shape[0], st_vel.shape[0])
    y = (slope*(x-x_1))+y_1


    
    plt.figure()
    plt.plot(y, color = 'Red')
    plt.imshow(st_vel, cmap='viridis')
    plt.title(r'$V_{stellar}$')
    plt.gca().invert_yaxis()
    plt.colorbar(orientation='vertical',label = r'$km/s$')
    plt.savefig(data+'_st_vel.png')

    plt.figure()
    plt.plot(y, color = 'Red')
    plt.imshow(oiii_ew, cmap='viridis')
    plt.title(r'$EW_{OIII}$')
    plt.gca().invert_yaxis()
    plt.colorbar(orientation='vertical',label = r'$log(\AA)$')
    plt.savefig(data+'_oiii.png')



[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/dapall-v3_1_1-3.1.0.fits' mode='rb' closefd=True> (ResourceWarning)


In [50]:
SAMPLE_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul_2 = fits.open(SAMPLE_PATH)
hdu_2 = hdul_2[1].data
plateifu_2 = hdu_2['PLATEIFU']


for data in tqdm(plateifu_2):
    if len(data) >= 4:
        plot_major_axis(data)
    else:
        pass

100%|██████████| 173/173 [09:42<00:00,  3.37s/it]


In [112]:
def plot_major_axis(data):
    

    dapall_index = list(plateifu).index(data)
    p_slope = np.tan(petro_angle[dapall_index])
    s_slope = np.tan(sersic_angle[dapall_index])

    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    st_vel = maps.stellar_vel.value
    oiii_ew = maps.emline_gew_oiii_5008.value
    phi = maps.spx_ellcoo_elliptical_azimuth.value
    r_re = maps.spx_ellcoo_r_re.value
    
    axis_indicat = np.zeros(phi.shape)

    for i in range(phi.shape[0]):
        for j in range(phi.shape[1]):
            if phi[i][j] >= 88 and phi[i][j] <= 92:
                axis_indicat[i][j] = 1
            elif phi[i][j] >= 268 and phi[i][j] <= 272:
                axis_indicat[i][j] = 1
            elif r_re[i][j] >= 0.9 and r_re[i][j] <=1.1:
                axis_indicat[i][j] = 1
            else:
                pass
            
            
            
            
    plt.figure()
    plt.imshow(st_vel, cmap='viridis')
    plt.imshow(axis_indicat, 'Greys', alpha = 0.3)
    plt.title(r'$V_{stellar}$')
    plt.gca().invert_yaxis()
    plt.colorbar(orientation='vertical',label = r'$km/s$')
    plt.savefig(data+'.png')
    


In [2]:
def marvin_major_axis(data):
    
    dapall_index = list(plateifu).index(data)
    p_slope = np.tan(petro_angle[dapall_index])
    s_slope = np.tan(sersic_angle[dapall_index])

    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    st_vel = maps.stellar_vel
    #oiii_ew = maps.emline_gew_oiii_5008.value
    phi = maps.spx_ellcoo_elliptical_azimuth.value
    r_re = maps.spx_ellcoo_r_re.value

    axis_indicat = np.zeros(phi.shape)

    for i in range(phi.shape[0]):
        for j in range(phi.shape[1]):
            if phi[i][j] >= 88 and phi[i][j] <= 92:
                axis_indicat[i][j] = 1
            elif phi[i][j] >= 268 and phi[i][j] <= 272:
                axis_indicat[i][j] = 1
            elif r_re[i][j] >= 0.9 and r_re[i][j] <=1.1:
                axis_indicat[i][j] = 1
            else:
                pass


    st_vel.plot(snr_min = None)
    plt.imshow(axis_indicat, 'Greys', alpha = 0.3)
    plt.savefig(data+'.png')

## Multi-processing


if __name__ == '__main__':
    pool = mp.Pool(processes=4)

    pool.imap_unordered(marvin_major_axis, plateifu_2):
        results.append(result)
       
    print(results)

In [5]:
## Plotting

SAMPLE_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul_2 = fits.open(SAMPLE_PATH)
hdu_2 = hdul_2[1].data
plateifu_2 = hdu_2['PLATEIFU']


for i in plateifu_2:
    marvin_major_axis(i)

[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/bicone_training_set.fits' mode='rb' closefd=True> (ResourceWarning)
[WARNING]: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). (RuntimeWarning)


# Ploting axis over image

In [145]:
def image_major_axis(data):

    dapall_index = list(plateifu).index(data)
    p_slope = np.tan(petro_angle[dapall_index])
    s_slope = np.tan(sersic_angle[dapall_index])

    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    st_vel = maps.stellar_vel
    #oiii_ew = maps.emline_gew_oiii_5008.value
    phi = maps.spx_ellcoo_elliptical_azimuth.value
    r_re = maps.spx_ellcoo_r_re.value

    axis_indicat = np.zeros(phi.shape)

    for i in range(phi.shape[0]):
        for j in range(phi.shape[1]):
            if phi[i][j] >= 88 and phi[i][j] <= 92:
                axis_indicat[i][j] = 100
            elif phi[i][j] >= 268 and phi[i][j] <= 272:
                axis_indicat[i][j] = 100
            elif r_re[i][j] >= 0.9 and r_re[i][j] <=1.1:
                axis_indicat[i][j] = 100
            else:
                pass
            
    im = Image(data)

    spaxel_len = float(str(im.wcs)[108:114]) * 2
    factor = spaxel_len / axis_indicat.shape[0]

    large_mask = ndimage.zoom(axis_indicat, factor, order=0)
    # large_mask[large_mask > 0] = 1


    mask = np.flipud(large_mask)
    im = showImage(plateifu = data)
    masked = np.ma.masked_where(mask == 0, mask)

    plt.imshow(im, interpolation='none')
    plt.imshow(masked, 'autumn', interpolation='none', alpha=0.7)
    plt.savefig(data+'_image.png')


In [146]:
## Plotting

SAMPLE_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul_2 = fits.open(SAMPLE_PATH)
hdu_2 = hdul_2[1].data
plateifu_2 = hdu_2['PLATEIFU']


for i in plateifu_2:
    image_major_axis(i)

[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/bicone_training_set.fits' mode='rb' closefd=True> (ResourceWarning)
[WARNING]: subprocess 12895 is still running (ResourceWarning)
[WARNING]: subprocess 12901 is still running (ResourceWarning)


MarvinError: something went wrong. Error is: Failed to parse input name 7960-6103: Cannot release a lock that's no longer owned.
You can submit this error to Marvin GitHub Issues (https://github.com/sdss/marvin/issues/new).
Fill out a subject and some text describing the error that just occurred.
If able, copy and paste the full traceback information into the issue as well.